# 07 · Entrega Final (Colab + Kaggle API)
Este cuaderno está preparado para ejecutarse en **Google Colab con GPU** e incluye integración con **Kaggle API** para descargar automáticamente el dataset de **PetFinder Adoption Prediction**.


In [ ]:
# miro que anda, sino sigue

!nvidia-smi || echo "No se detectó GPU (si estás local está OK)"


Thu Aug 28 22:33:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# copie y pegue de siempre
import os
running_in_colab = 'google.colab' in str(get_ipython())
if running_in_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Drive montado en /content/drive")
else:
    print("No estás en Colab; se omite el montaje de Drive.")


Mounted at /content/drive
Drive montado en /content/drive


In [ ]:
# las dependencias
!pip -q install --upgrade pip
!pip -q install pandas numpy scikit-learn matplotlib pillow tqdm optuna lightgbm xgboost albumentations timm kaggle
# Si necesitas una versión concreta de torch/cuDNN, descomenta y ajusta:
# !pip -q install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 54.1 MB/s eta 0:00:00


### 3.1) Autenticación Kaggle (dos opciones)
- **Opción A (recomendada):** Guardá tu `kaggle.json` en Drive y apunta la ruta.
- **Opción B:** Subí manualmente el archivo `kaggle.json` desde tu PC.


In [ ]:
# llamo a Kaggle

import os, shutil, pathlib

running_in_colab = 'google.colab' in str(get_ipython())
KAGGLE_JSON_FROM_DRIVE = "/content/drive/MyDrive/LaboII/kaggle.json"

if running_in_colab and os.path.exists(KAGGLE_JSON_FROM_DRIVE):
    os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
    shutil.copy(KAGGLE_JSON_FROM_DRIVE, os.path.expanduser("~/.kaggle/kaggle.json"))
    os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"), 0o600)
    print("kaggle.json copiado desde Drive ✔")
else:
    print("No se encontró kaggle.json en Drive. Podés usar la Opción B (subida manual) debajo.")


kaggle.json copiado desde Drive ✔


In [ ]:
# Descarga de datos PetFinder desde Kaggle SOLO LA PRIMERA VEZ
import os, pathlib, zipfile, subprocess

running_in_colab = 'google.colab' in str(get_ipython())
if running_in_colab:
    # Carpeta base del proyecto en Colab/Drive (ajusta si querés otra estructura)
    PROJECT_DIR = "/content/drive/MyDrive/LaboII"
    DATA_DIR = f"{PROJECT_DIR}/input/petfinder-adoption-prediction"
else:
    PROJECT_DIR = ".."
    DATA_DIR = "../input/petfinder-adoption-prediction"

pathlib.Path(DATA_DIR).mkdir(parents=True, exist_ok=True)

# Usamos la Kaggle CLI para descargar la competencia oficial
# Nota: Es necesario haber aceptado las reglas en Kaggle previamente.
print("Descargando (o validando) archivos de Kaggle…")
!kaggle competitions download -c petfinder-adoption-prediction -p "$DATA_DIR" -q

# Descomprimir todos los ZIPs que aún no se hayan extraído
for p in pathlib.Path(DATA_DIR).glob("*.zip"):
    dest_marker = p.with_suffix(".unzipped.marker")
    if not dest_marker.exists():
        print(f"Descomprimiendo {p.name} …")
        with zipfile.ZipFile(p, 'r') as z:
            z.extractall(DATA_DIR)
        dest_marker.write_text("ok")
    else:
        print(f"{p.name} ya estaba descomprimido.")
print("Listo.")


In [ ]:

import os, pathlib


# 2) Fijar carpeta del proyecto (ajusta si tu carpeta se llama diferente)
PROJECT_DIR = "/content/drive/MyDrive/LaboII"
DATA_DIR    = os.path.join(PROJECT_DIR, "input", "petfinder-adoption-prediction")

# 3) Comprobar existencia de archivos clave
train_csv = os.path.join(DATA_DIR, "train", "train.csv")
test_csv  = os.path.join(DATA_DIR, "test", "test.csv")
paths = [train_csv, test_csv,
         os.path.join(DATA_DIR, "train_images"),
         os.path.join(DATA_DIR, "test_images")]

print("PROJECT_DIR:", PROJECT_DIR)
print("DATA_DIR   :", DATA_DIR)
for p in paths:
    print(p, "EXISTE?", os.path.exists(p))

# 4) Solo si NO existe, descargar con Kaggle (opcional)
if not os.path.exists(train_csv):
    print("⚠️ No se encontró train.csv, intentando descargar con Kaggle…")
    !kaggle competitions download -c petfinder-adoption-prediction -p "$DATA_DIR" -q
    for z in pathlib.Path(DATA_DIR).glob("*.zip"):
        print("Descomprimiendo", z.name)
        import zipfile
        with zipfile.ZipFile(z, 'r') as zip_ref:
            zip_ref.extractall(DATA_DIR)
else:
    print("✅ Dataset ya presente en Drive, usando los datos locales")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
PROJECT_DIR: /content/drive/MyDrive/LaboII
DATA_DIR   : /content/drive/MyDrive/LaboII/input/petfinder-adoption-prediction
/content/drive/MyDrive/LaboII/input/petfinder-adoption-prediction/train/train.csv EXISTE? True
/content/drive/MyDrive/LaboII/input/petfinder-adoption-prediction/test/test.csv EXISTE? True
/content/drive/MyDrive/LaboII/input/petfinder-adoption-prediction/train_images EXISTE? True
/content/drive/MyDrive/LaboII/input/petfinder-adoption-prediction/test_images EXISTE? True
✅ Dataset ya presente en Drive, usando los datos locales


In [ ]:
# aca que dios nos ayude y chatgpt, porque? porque cada un define sus rutas... para mi con esta debe andar
import os

PATH_TO_TRAIN = os.path.join(DATA_DIR, "train", "train.csv")
PATH_TO_TEST  = os.path.join(DATA_DIR, "test", "test.csv")
PATH_TO_IMAGES_DIR = os.path.join(DATA_DIR, "train_images")
PATH_TO_TEST_IMAGES_DIR = os.path.join(DATA_DIR, "test_images")

print("PROJECT_DIR:", PROJECT_DIR)
print("DATA_DIR:", DATA_DIR)
for p in [PATH_TO_TRAIN, PATH_TO_IMAGES_DIR, PATH_TO_TEST, PATH_TO_TEST_IMAGES_DIR]:
    print(p, "EXISTE?", os.path.exists(p))


PROJECT_DIR: /content/drive/MyDrive/LaboII
DATA_DIR: /content/drive/MyDrive/LaboII/input/petfinder-adoption-prediction
/content/drive/MyDrive/LaboII/input/petfinder-adoption-prediction/train/train.csv EXISTE? True
/content/drive/MyDrive/LaboII/input/petfinder-adoption-prediction/train_images EXISTE? True
/content/drive/MyDrive/LaboII/input/petfinder-adoption-prediction/test/test.csv EXISTE? True
/content/drive/MyDrive/LaboII/input/petfinder-adoption-prediction/test_images EXISTE? True


In [ ]:
# 6) Verificación rápida de PyTorch + CUDA (esta linea es toda de chatgpt... solamente de chequeo)
try:
    import torch
    print("torch:", torch.__version__)
    print("CUDA disponible?:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("GPU:", torch.cuda.get_device_name(0))
except Exception as e:
    print("No se pudo importar torch todavía:", e)


torch: 2.8.0+cu126
CUDA disponible?: True
GPU: Tesla T4


# Entrega Final Laboratorio de implementación 2




In [ ]:
!pip install --quiet \
    optuna[all] \
    lightgbm \
    xgboost \
    albumentations \
    timm \
    plotly

    #aca llame a optuma y lo que se neceita


## Librerías

In [ ]:
#Import de librerias basicas tablas y matrices
import numpy as np
import pandas as pd

#Gradient Boosting
import lightgbm as lgb

#Funciones auxiliares sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold #Split y cross Validation
from sklearn.metrics import cohen_kappa_score, accuracy_score, balanced_accuracy_score #Metricas
from sklearn.utils import shuffle

#Visualizacióon
from plotly import express as px


import os

#Optimizacion de hiperparametros
import optuna
from optuna.artifacts import FileSystemArtifactStore, upload_artifact

#Guardado de objetos en archivos joblib
from joblib import load, dump

# Para análisis de texto
import re


In [ ]:


# 2) Cargar *por ruta* tu utils.py del Drive bajo otro nombre de módulo
import os, importlib.util

PATH_WORK = "/content/drive/MyDrive/LaboII/work"
UTILS_FILE = os.path.join(PATH_WORK, "utils.py")   # o el nombre que le diste

spec = importlib.util.spec_from_file_location("utils_drive", UTILS_FILE)
utils_drive = importlib.util.module_from_spec(spec)
spec.loader.exec_module(utils_drive)

# 3) Tomá las funciones desde ese módulo
plot_confusion_matrix   = utils_drive.plot_confusion_matrix
get_artifact_filename   = utils_drive.get_artifact_filename

# 4) Verificación
print("utils cargado desde:", utils_drive.__file__)



utils cargado desde: /content/drive/MyDrive/LaboII/work/utils.py


## Paths del repo

In [ ]:

#Subimos dos niveles para quedar en la carpeta que contiene input y UA_MDM_Labo2
BASE_DIR = '../'

#Datos de entrenamiento
PATH_TO_TRAIN = os.path.join(BASE_DIR, "input/petfinder-adoption-prediction/train/train.csv")

#Salida de modelos entrenados
PATH_TO_MODELS = os.path.join(BASE_DIR, "work/models")

#Artefactos a subir a optuna
PATH_TO_TEMP_FILES = os.path.join(BASE_DIR, "work/optuna_temp_artifacts")

#Artefactos que optuna gestiona
PATH_TO_OPTUNA_ARTIFACTS = os.path.join(BASE_DIR, "work/optuna_artifacts")


SEED = 20001101 #Semilla de procesos aleatorios (para poder replicar exactamente al volver a correr un modelo)
TEST_SIZE = 0.2 #Facción para train/test= split

## Lectura de los datasets

In [ ]:
# Forzar rutas correctas
DATA_DIR = "/content/drive/MyDrive/LaboII/input/petfinder-adoption-prediction"
PATH_TO_TRAIN = os.path.join(DATA_DIR, "train", "train.csv")
PATH_TO_TEST  = os.path.join(DATA_DIR, "test", "test.csv")
PATH_TO_IMAGES_DIR = os.path.join(DATA_DIR, "train_images")
PATH_TO_TEST_IMAGES_DIR = os.path.join(DATA_DIR, "test_images")

print("PATH_TO_TRAIN =", PATH_TO_TRAIN, "EXISTE?", os.path.exists(PATH_TO_TRAIN))
assert os.path.exists(PATH_TO_TRAIN), "No se encuentra train.csv en la ruta esperada"


PATH_TO_TRAIN = /content/drive/MyDrive/LaboII/input/petfinder-adoption-prediction/train/train.csv EXISTE? True


In [ ]:
print("PATH_TO_TRAIN =", PATH_TO_TRAIN)
dataset = pd.read_csv(PATH_TO_TRAIN)
print(dataset.shape)



PATH_TO_TRAIN = /content/drive/MyDrive/LaboII/input/petfinder-adoption-prediction/train/train.csv
(14993, 24)


## Datos tabulares

### Feature engineering manual

In [ ]:
### Se crea la variable Has_name
# Lista de nombres genéricos que NO consideramos verdaderos nombres
nombres_invalidos = ['Unnamed', 'No name', 'No Name Yet', 'Urgent', 'Lost Dog']

# Rellenamos nulos
dataset['Name'] = dataset['Name'].fillna('Unnamed')

# Identificamos si contienen dígitos
contiene_numeros = dataset['Name'].str.contains(r'\d', regex=True)

# Extension del nombre
extension = (dataset['Name'].str.len() <= 3) | (dataset['Name'].str.len() > 15)

# Creamos la columna Has_name: 1 si no es inválido y no contiene números
dataset['Has_name'] = (~dataset['Name'].isin(nombres_invalidos) & ~contiene_numeros & ~extension).astype(int)

### Largo de la descripcion
dataset['Description_length'] = dataset['Description'].str.len()

### Score de salud
dataset['Cuidado_Medico'] = (
    (dataset['Vaccinated'] == 1).astype(int) +
    (dataset['Dewormed'] == 1).astype(int) +
    (dataset['Sterilized'] == 1).astype(int)
)

### Edad
def clasificar_edad(age):
    if age <= 6:
        return 1  # Cachorro
    elif age > 72:
        return 4 # Adulto Mayor
    elif age > 24:
        return 3  # Adulto
    else:
        return 2  # Normal

dataset['Age_group'] = dataset['Age'].apply(clasificar_edad)


### Raza
dataset['Raza_pura'] = np.where(
    (dataset['Breed1'] != 307) & (dataset['Breed2'] == 0), 1, 0
)

### Cantidad de mascotas a adoptar
def clasificar_cantidad(q):
    if q == 1:
        return 0
    elif q <= 3:
        return 1
    elif q <= 6:  # Cambiado a <= 6 para incluir hasta 6
        return 2
    else:
        return 3

dataset['Cant_Adopcion'] = dataset['Quantity'].apply(clasificar_cantidad)

### Color
dataset['color_count'] = (
    (dataset['Color1'] != 0).astype(int) +
    (dataset['Color2'] != 0).astype(int) +
    (dataset['Color3'] != 0).astype(int)
)

### Adopcion gratuita
dataset['gratis'] = (dataset['Fee'] == 0).astype(int)

### Genero y castracion
dataset['Is_female_intact'] = (dataset['Gender'] == 2) & (dataset['Sterilized'] == 2)
dataset['Is_male_intact'] = (dataset['Gender'] == 1) & (dataset['Sterilized'] == 2)


### RescuerID repetidos
dataset['Rescuer_repetido'] = dataset.duplicated('RescuerID', keep=False).astype(int)

### Gato y Perro segun genero
dataset['Dog_male'] = ((dataset['Type'] == 1) & (dataset['Gender'] == 1)).astype(int)
dataset['Cat_male'] = ((dataset['Type'] == 2) & (dataset['Gender'] == 1)).astype(int)

### Hay videos
dataset['Video'] = (dataset['VideoAmt'] > 1).astype(int)

### Cantidad de fotos
def clasificar_fotos(x):
    if x == 0:
        return 0
    elif x <= 5:
        return 1
    else:
        return 2

dataset['Fotos'] = dataset['PhotoAmt'].apply(clasificar_fotos)

### Indica si la descripción contiene una palabra "linda"
# Lista de palabras clave (en minúsculas)
palabras_lindas = [
    'love', 'loving', 'sweet', 'friendly', 'cute', 'adorable',
    'affectionate', 'gentle', 'cuddly', 'playful', 'happy', 'kind',
    'compassionate', 'endearing', 'charming', 'delightful', 'joyful', 'precious']

# Expresión regular que detecta si alguna palabra está presente
patron = r'\b(' + '|'.join(palabras_lindas) + r')\b'

# Crear variable binaria
dataset['Descripcion_linda'] = dataset['Description'].str.lower().str.contains(patron, flags=re.IGNORECASE, na=False).astype(int)


### Combinacion de colores
def sorted_color_combo(row):
    colores = sorted([row['Color1'], row['Color2'], row['Color3']])
    return int(''.join(map(str, colores)))

dataset['Color_combo'] = dataset.apply(sorted_color_combo, axis=1)

/tmp/ipython-input-949666915.py:106: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  dataset['Descripcion_linda'] = dataset['Description'].str.lower().str.contains(patron, flags=re.IGNORECASE, na=False).astype(int)


In [ ]:
df_final = dataset

### Feature Engineering con Random Forest

#### Opción one-hot encoding

In [ ]:
""" import pandas as pd
from sklearn.ensemble import RandomForestClassifier

X = dataset.drop(columns=['Description', 'RescuerID', 'PetID', 'Name', 'AdoptionSpeed'])
y = dataset['AdoptionSpeed']

# Parámetros
n_modelos = 20       # cantidad de Random Forests
n_arboles = 1       # árboles por modelo
n_hojas = 16        # Hojas por arbol

# Copia del DataFrame original
df_final = X.copy()

# Entrenar y agregar columnas
for i in range(1, n_modelos + 1):
    rf = RandomForestClassifier(
        n_estimators=n_arboles,
        max_leaf_nodes= n_hojas,
        random_state=i
    )
    rf.fit(X, y)

    # Obtener la hoja final de cada observación
    leaf_ids = rf.apply(X).ravel()  # Devuelve un ID de hoja por fila

    # Obtener los IDs únicos de hojas (puede ser menos de 16 si no se usan todas)
    unique_leaves = sorted(set(leaf_ids))

    # DataFrame con columnas 0/1 por hoja
    leaf_df = pd.DataFrame(
        {f"R{i}L_{j+1}": (leaf_ids == leaf).astype(int)
            for j, leaf in enumerate(unique_leaves)},
        index=X.index
    )

    # Agregar al DataFrame final
    df_final = pd.concat([df_final, leaf_df], axis=1)

# Ahora df_final contiene las variables originales + las columnas RFxLy se le suma la clase
df_final = pd.concat([df_final, y], axis=1)
df_final.head() """

' import pandas as pd\nfrom sklearn.ensemble import RandomForestClassifier\n\nX = dataset.drop(columns=[\'Description\', \'RescuerID\', \'PetID\', \'Name\', \'AdoptionSpeed\'])\ny = dataset[\'AdoptionSpeed\']\n\n# Parámetros\nn_modelos = 20       # cantidad de Random Forests\nn_arboles = 1       # árboles por modelo\nn_hojas = 16        # Hojas por arbol\n\n# Copia del DataFrame original\ndf_final = X.copy()\n\n# Entrenar y agregar columnas\nfor i in range(1, n_modelos + 1):\n    rf = RandomForestClassifier(\n        n_estimators=n_arboles,\n        max_leaf_nodes= n_hojas,\n        random_state=i\n    )\n    rf.fit(X, y)\n\n    # Obtener la hoja final de cada observación\n    leaf_ids = rf.apply(X).ravel()  # Devuelve un ID de hoja por fila\n\n    # Obtener los IDs únicos de hojas (puede ser menos de 16 si no se usan todas)\n    unique_leaves = sorted(set(leaf_ids))\n\n    # DataFrame con columnas 0/1 por hoja\n    leaf_df = pd.DataFrame(\n        {f"R{i}L_{j+1}": (leaf_ids == leaf).a

#### Opción leaf_id

In [ ]:
""" import pandas as pd
from sklearn.ensemble import RandomForestClassifier

X = dataset.drop(columns=['Description', 'RescuerID', 'PetID', 'Name', 'AdoptionSpeed'])
y = dataset['AdoptionSpeed']

# Parámetros
n_modelos = 50       # cantidad de Random Forests
n_arboles = 1       # árboles por modelo
n_hojas = 16        # Hojas por arbol

# Copia del DataFrame original
df_final = X.copy()

# Entrenar y agregar columnas
for i in range(1, n_modelos + 1):
    rf = RandomForestClassifier(
        n_estimators=n_arboles,
        max_leaf_nodes= n_hojas,
        random_state=i
    )
    rf.fit(X, y)

    # Obtener la hoja final de cada observación
    leaf_ids = rf.apply(X).ravel()  # Devuelve un ID de hoja por fila

    # Crear nombres de columnas según formato RF{i}L{j}
    colnames = [f"RF{i}"]

    # Convertir a DataFrame
    leaf_df = pd.DataFrame(leaf_ids, columns=colnames, index=X.index)

    # Agregar al DataFrame final
    df_final = pd.concat([df_final, leaf_df], axis=1)

# Ahora df_final contiene las variables originales + las columnas RFxLy le sumo la clase
df_final = pd.concat([df_final, y], axis=1)
df_final.head() """

' import pandas as pd\nfrom sklearn.ensemble import RandomForestClassifier\n\nX = dataset.drop(columns=[\'Description\', \'RescuerID\', \'PetID\', \'Name\', \'AdoptionSpeed\'])\ny = dataset[\'AdoptionSpeed\']\n\n# Parámetros\nn_modelos = 50       # cantidad de Random Forests\nn_arboles = 1       # árboles por modelo\nn_hojas = 16        # Hojas por arbol\n\n# Copia del DataFrame original\ndf_final = X.copy()\n\n# Entrenar y agregar columnas\nfor i in range(1, n_modelos + 1):\n    rf = RandomForestClassifier(\n        n_estimators=n_arboles,\n        max_leaf_nodes= n_hojas,\n        random_state=i\n    )\n    rf.fit(X, y)\n\n    # Obtener la hoja final de cada observación\n    leaf_ids = rf.apply(X).ravel()  # Devuelve un ID de hoja por fila\n\n    # Crear nombres de columnas según formato RF{i}L{j}\n    colnames = [f"RF{i}"]\n\n    # Convertir a DataFrame\n    leaf_df = pd.DataFrame(leaf_ids, columns=colnames, index=X.index)\n\n    # Agregar al DataFrame final\n    df_final = pd.conca

### Selección de variables y división en Train y Test

In [ ]:
# Features menos estas variables

var_to_remove = ['Description', 'RescuerID', 'PetID', 'Name', 'AdoptionSpeed']

features = [f for f in df_final.columns if f not in var_to_remove]

label = 'AdoptionSpeed'


In [ ]:
#Separo un 20% para test estratificado opr target
train, test = train_test_split(df_final,
                               test_size = TEST_SIZE,
                               random_state = SEED,
                               stratify = df_final.AdoptionSpeed)

In [ ]:
#Genero dataframes de train y test con sus respectivos targets
X_train = train[features]
y_train = train[label]

X_test = test[features]
y_test = test[label]

### Búsqueda mejores hiperparámetros


In [ ]:
import matplotlib.pyplot as plt  # <- necesario para plt.close

def cv_es_lgb_objective(trial):
    lgb_params = {
        'objective': 'multiclass',
        'verbosity': -1,
        'num_class': len(y_train.unique()),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.003, 0.1, log=True),
    }

    scores_ensemble = np.zeros((len(y_test), len(y_train.unique())))
    score_folds = 0
    skf = StratifiedKFold(n_splits=5)

    for i, (if_index, oof_index) in enumerate(skf.split(X_train, y_train)):
        lgb_if_dataset = lgb.Dataset(X_train.iloc[if_index], label=y_train.iloc[if_index], free_raw_data=False)
        lgb_oof_dataset = lgb.Dataset(X_train.iloc[oof_index], label=y_train.iloc[oof_index], free_raw_data=False)

        lgb_model = lgb.train(
            lgb_params,
            lgb_if_dataset,
            valid_sets=lgb_oof_dataset,
            num_boost_round=5000,                         # <- en train va acá
            callbacks=[lgb.early_stopping(10, verbose=False)],
            feval=lgb_custom_metric_kappa
        )

        scores_ensemble += lgb_model.predict(X_test)
        score_folds += cohen_kappa_score(
            y_train.iloc[oof_index],
            lgb_model.predict(X_train.iloc[oof_index]).argmax(axis=1),
            weights='quadratic'
        ) / 5

    # ----- guardar artefactos -----
    predicted_filename = os.path.join(PATH_TO_TEMP_FILES, f'test_{trial.study.study_name}_{trial.number}.joblib')
    predicted_df = test.copy()
    predicted_df['pred'] = [scores_ensemble[p, :] for p in range(scores_ensemble.shape[0])]
    dump(predicted_df, predicted_filename)
    upload_artifact(study_or_trial=trial, file_path=predicted_filename, artifact_store=artifact_store)

    cm_filename = os.path.join(PATH_TO_TEMP_FILES, f'cm_{trial.study.study_name}_{trial.number}.png')
    fi_filename = os.path.join(PATH_TO_TEMP_FILES, f'fi_{trial.study.study_name}_{trial.number}.png')

    fig_cm = plot_confusion_matrix(
        y_test,
        scores_ensemble.argmax(axis=1),
        labels=np.arange(len(np.unique(y_train))),
        title="Matriz de confusión",
        counts=True
    )
    fig_cm.write_image(cm_filename, scale=2)  # requiere kaleido

    ax = lgb.plot_importance(lgb_model, importance_type="gain", figsize=(10, 7))
    ax.figure.savefig(fi_filename, bbox_inches="tight", dpi=150)
    plt.close(ax.figure)

    upload_artifact(study_or_trial=trial, file_path=cm_filename, artifact_store=artifact_store)
    upload_artifact(study_or_trial=trial, file_path=fi_filename, artifact_store=artifact_store)

    # métrica extra en test
    test_score = cohen_kappa_score(y_test, scores_ensemble.argmax(axis=1), weights='quadratic')
    trial.set_user_attr("test_score", test_score)

    return score_folds


In [ ]:
# esto hice porque en el entorno de colap no tengo forma de traer cv_es_lgb_objective, que es un embole tenerlo... esta solucion la codee con ayuda de chatgpt, porlo que esta sujeto a pruebas
# no es necesario cargar de nuevos todas las utilidades.....

import numpy as np, os, matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold
from joblib import dump

NUM_CLASS = 5  # o len(y_train.unique())


def lgb_custom_metric_kappa(preds, train_data):
    """feval para LightGBM (multiclase) usando Quadratic Kappa."""
    y_true = train_data.get_label().astype(int)
    # LightGBM entrega preds como vector 1D de largo n * num_class (orden por clase)
    if preds.ndim == 1:
        probs = preds.reshape(NUM_CLASS, -1).T  # (n, num_class)
    else:
        probs = preds  # por si ya viniera (n, num_class)
    y_pred = probs.argmax(axis=1)
    value = cohen_kappa_score(y_true, y_pred, weights="quadratic")
    return "kappa", float(value), True

# === 2) OBJETIVO DE OPTUNA (en la MISMA celda) ===
def cv_es_lgb_objective(trial):
    params = {
        "objective": "multiclass",
        "num_class": NUM_CLASS,
        "verbosity": -1,
        # hiperparámetros a optimizar
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "learning_rate": trial.suggest_float("learning_rate", 0.003, 0.1, log=True),
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores_ensemble = np.zeros((len(y_test), NUM_CLASS))
    score_cv = 0.0

    for if_idx, oof_idx in skf.split(X_train, y_train):
        dtrain = lgb.Dataset(X_train.iloc[if_idx], label=y_train.iloc[if_idx], free_raw_data=False)
        dvalid = lgb.Dataset(X_train.iloc[oof_idx], label=y_train.iloc[oof_idx], free_raw_data=False)

        booster = lgb.train(
            params,
            dtrain,
            valid_sets=[dvalid],
            num_boost_round=5000,
            callbacks=[lgb.early_stopping(10, verbose=False)],
            feval=lgb_custom_metric_kappa,
        )

        # acumulo probs en test para ensamblar
        scores_ensemble += booster.predict(X_test)
        # kappa en OOF
        oof_pred = booster.predict(X_train.iloc[oof_idx]).argmax(axis=1)
        score_cv += cohen_kappa_score(y_train.iloc[oof_idx], oof_pred, weights="quadratic") / 5

    # ----- artefactos (opcional) -----
    try:
        from utils import plot_confusion_matrix  # por si no estaba importado aún
        cm_path = os.path.join(PATH_TO_TEMP_FILES, f"cm_{trial.study.study_name}_{trial.number}.png")
        fig_cm = plot_confusion_matrix(
            y_test, scores_ensemble.argmax(axis=1),
            labels=np.arange(NUM_CLASS), title="Matriz de confusión", counts=True
        )
        fig_cm.write_image(cm_path, scale=2)
        from optuna.artifacts import upload_artifact
        upload_artifact(study_or_trial=trial, file_path=cm_path, artifact_store=artifact_store)
    except Exception as e:
        print("Aviso al generar/adjuntar CM:", e)

    # métrica extra en test
    test_score = cohen_kappa_score(y_test, scores_ensemble.argmax(axis=1), weights="quadratic")
    trial.set_user_attr("test_score", float(test_score))

    return float(score_cv)


In [ ]:
study.optimize(cv_es_lgb_objective, n_trials=3)  # o más


[I 2025-08-28 22:36:53,545] Trial 3 finished with value: 0.29990712200663616 and parameters: {'lambda_l1': 0.002004962934913646, 'lambda_l2': 4.8551439057890526e-08, 'num_leaves': 232, 'feature_fraction': 0.6690374288841778, 'bagging_fraction': 0.9866716003446006, 'bagging_freq': 4, 'min_child_samples': 48, 'learning_rate': 0.003398775834786453}. Best is trial 2 with value: 0.3518550349723765.


Aviso al generar/adjuntar CM: No module named 'utils'


[I 2025-08-28 22:36:56,706] Trial 4 finished with value: 0.25990504213849286 and parameters: {'lambda_l1': 0.009046802805830815, 'lambda_l2': 0.04276614948554251, 'num_leaves': 4, 'feature_fraction': 0.5871518257576716, 'bagging_fraction': 0.8619769561481219, 'bagging_freq': 4, 'min_child_samples': 64, 'learning_rate': 0.04152798178580237}. Best is trial 2 with value: 0.3518550349723765.


Aviso al generar/adjuntar CM: No module named 'utils'


[I 2025-08-28 22:37:10,499] Trial 5 finished with value: 0.3429916426585729 and parameters: {'lambda_l1': 4.095545574161033e-05, 'lambda_l2': 0.0009021448485282158, 'num_leaves': 170, 'feature_fraction': 0.4893457992586764, 'bagging_fraction': 0.9106205275754494, 'bagging_freq': 5, 'min_child_samples': 59, 'learning_rate': 0.021471574594316305}. Best is trial 2 with value: 0.3518550349723765.


Aviso al generar/adjuntar CM: No module named 'utils'


In [ ]:

import os, optuna
from optuna.artifacts import FileSystemArtifactStore, upload_artifact

PATH_WORK = "/content/drive/MyDrive/LaboII/work"
DB_PATH   = os.path.join(PATH_WORK, "db.sqlite3")
PATH_TO_OPTUNA_ARTIFACTS = os.path.join(PATH_WORK, "optuna_artifacts")
PATH_TO_TEMP_FILES       = os.path.join(PATH_WORK, "optuna_temp_artifacts")
os.makedirs(PATH_TO_OPTUNA_ARTIFACTS, exist_ok=True)
os.makedirs(PATH_TO_TEMP_FILES, exist_ok=True)

storage_url = f"sqlite:///{DB_PATH}"
artifact_store = FileSystemArtifactStore(base_path=PATH_TO_OPTUNA_ARTIFACTS)

STUDY_NAME = "3 - FE Manual"

study = optuna.create_study(
    direction="maximize",
    storage=storage_url,
    study_name=STUDY_NAME,
    load_if_exists=True,
)
print("Estudio:", study.study_name, "| trials totales:", len(study.trials))


[I 2025-08-28 22:37:10,563] Using an existing study with name '3 - FE Manual' instead of creating a new one.


Estudio: 3 - FE Manual | trials totales: 6


### Modelo Final

In [ ]:
#Vamos a replicar el resultado de la optimizacion reentrenando el modelo con el mejor conjunto de hiperparametros
#Generamos parametros incluyendo los fijos y la mejor solución que encontro optuna
lgb_params =  {
                        'objective': 'multiclass',
                        'verbosity':-1,
                        'num_class': len(y_train.unique())} | study.best_params

lgb_train_dataset = lgb.Dataset(data=X_train,
                                label=y_train)


#Entreno
lgb_model = lgb.train(lgb_params,
                    lgb_train_dataset)

#Muestro matriz de confusion y kappa
display(plot_confusion_matrix(y_test,lgb_model.predict(X_test).argmax(axis=1)))

cohen_kappa_score(y_test,lgb_model.predict(X_test).argmax(axis=1),
                             weights = 'quadratic')

/usr/local/lib/python3.12/dist-packages/kaleido/__init__.py:14: UserWarning:




This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.




np.float64(0.36881429701389534)

## Datos imágenes

### Librerías

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, cohen_kappa_score
import os
import shutil
import time
import copy
import datetime
from tqdm import tqdm

import optuna
from optuna.artifacts import FileSystemArtifactStore, upload_artifact

import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.nn.functional as F

from joblib import load, dump
# Verificamos que CUDA está funcional
torch.cuda.is_available()

True

### Modelo preentrenado (ResNet50)

In [ ]:
# Importo modelo ResNet entrenado en Imagenet
resnet50 = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
# Modificar la última capa para adaptarse a tu problema específico
num_ftrs = resnet50.fc.in_features
resnet50.fc = torch.nn.Linear(num_ftrs, 5) # Clasificación 5 clases
# Configuro para usar cuda si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet50 = resnet50.to(device)
# Instancio del criterio de pérdida CrossEntropyLoss
criterion = nn.CrossEntropyLoss()


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 125MB/s]


### Path

In [ ]:
import os

# Carpeta raíz del proyecto en tu Drive
PROJECT_DIR = "/content/drive/MyDrive/LaboII"
DATA_DIR    = os.path.join(PROJECT_DIR, "input/petfinder-adoption-prediction")

# Paths de dataset y artefactos
PATH_TO_TRAIN          = os.path.join(DATA_DIR, "train", "train.csv")
PATH_TO_IMAGES_DIR     = os.path.join(DATA_DIR, "train_images")
PATH_TO_TEMP_FILES     = os.path.join(PROJECT_DIR, "work", "optuna_temp_artifacts")
PATH_TO_OPTUNA_ARTIFACTS = os.path.join(PROJECT_DIR, "work", "optuna_artifacts")

# Modelo
MODEL_NAME    = '04 ResNet'
MODEL_VERSION = '1.0.0'

# Parámetros y variables
CREATE_PYTORCH_DIRECTORIES = 1
SEED        = 20001101
BATCH_SIZE  = 50
TEST_SIZE   = 0.2
IMAGE_SIZE  = 299
CPU_CORES   = os.cpu_count()

# Directorios de train/val para reorganizar imágenes por clase
new_train_directory = os.path.join(PROJECT_DIR, 'work/train_images_classes')
os.makedirs(new_train_directory, exist_ok=True)

new_val_directory   = os.path.join(PROJECT_DIR, 'work/val_images_classes')
os.makedirs(new_val_directory, exist_ok=True)

# Definir clases
class_names  = ['0', '1', '2', '3', '4']
class_to_idx = {class_name: i for i, class_name in enumerate(class_names)}

# Crear subcarpetas de clases
for clase in class_names:
    os.makedirs(os.path.join(new_train_directory, str(clase)), exist_ok=True)
    os.makedirs(os.path.join(new_val_directory, str(clase)), exist_ok=True)

print("PATH_TO_TRAIN:", PATH_TO_TRAIN, "EXISTE?", os.path.exists(PATH_TO_TRAIN))
print("PATH_TO_IMAGES_DIR:", PATH_TO_IMAGES_DIR, "EXISTE?", os.path.exists(PATH_TO_IMAGES_DIR))



PATH_TO_TRAIN: /content/drive/MyDrive/LaboII/input/petfinder-adoption-prediction/train/train.csv EXISTE? True
PATH_TO_IMAGES_DIR: /content/drive/MyDrive/LaboII/input/petfinder-adoption-prediction/train_images EXISTE? True


### Carga y Preprocesamiento

In [ ]:
def visualize_pet(pet_id):
    path_to_image = os.path.join(PATH_TO_IMAGES_DIR, f'{pet_id}-1.jpg') # Irá a la primera imagen de la mascota
    # Cargar la imagen
    image_to_show = cv2.imread(path_to_image)
    # Convertir a formato RGB
    image_to_show = cv2.cvtColor(image_to_show, cv2.COLOR_BGR2RGB)
    # Visualizar la imagen
    plt.imshow(image_to_show)
    plt.axis('off')  # No mostrar los ejes
    plt.show()

def visualize_image(image):
    # Convierte la imagen a un formato de enteros (CV_8U)
    image = cv2.convertScaleAbs(image)
    image= cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Visualizar la imagen
    plt.imshow(image.astype(np.uint8))
    plt.axis('off')  # No mostrar los ejes
    plt.show()

In [ ]:

if CREATE_PYTORCH_DIRECTORIES == 0: # Poner en 0 si ya tengo las carpetas train_images_classes y val_images_classes con las imágenes copiadas
    # Función para copiar las imágenes a los directorios correspondientes
    def copy_imag(data, directorio_destino):
        for index, row in data.iterrows():
            petID = row['PetID']
            adoption_speed = row['AdoptionSpeed']

            # Nombre del archivo de imagen
            nombre_archivo = f"{petID}-1.jpg"

            # Ruta completa de la imagen de origen
            ruta_origen = os.path.join(PATH_TO_IMAGES_DIR, nombre_archivo)

            # Ruta completa del directorio de destino
            ruta_destino = os.path.join(directorio_destino, str(adoption_speed), nombre_archivo)

            # Verificar si el archivo de origen existe
            if os.path.exists(ruta_origen):
                # Copiar el archivo de origen al directorio de destino
                shutil.copy2(ruta_origen, ruta_destino)
        print("Completada la copia a: ",str(directorio_destino))

    # Copiar las imágenes al directorio de train
    copy_imag(train, new_train_directory)

    # Copiar las imágenes al directorio de val
    copy_imag(test, new_val_directory)

    print("Proceso completado.")


In [ ]:
# Genero los DataLoaders
def create_dataloaders(train_directory, val_directory, batch_size, num_workers):
    # Transformaciones de imagen para el conjunto de entrenamiento
    train_transforms = transforms.Compose([
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.RandomHorizontalFlip(),
        # GaussianBlur con 50% de probabilidad
        transforms.RandomApply(
            [transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0))],
            p=0.5
        ),
        # ColorJitter con 50% de probabilidad
        transforms.RandomApply(
            [transforms.ColorJitter(brightness=0.2, contrast=0.2,
                                    saturation=0.2, hue=0.05)],
            p=0.5
        ),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

    # Transformaciones de imagen para el conjunto de validación (sin data augment)
    val_transforms = transforms.Compose([
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

    # Crear conjuntos de datos para el conjunto de entrenamiento y validación
    conjunto_entrenamiento = datasets.ImageFolder(train_directory, transform=train_transforms)
    conjunto_validacion = datasets.ImageFolder(val_directory, transform=val_transforms)

    # Asignar las clases ordenadas al conjunto de datos
    conjunto_entrenamiento.class_to_idx = {class_name: i for i, class_name in enumerate(class_names)}
    conjunto_validacion.class_to_idx = {class_name: i for i, class_name in enumerate(class_names)}

    # Crear dataloaders para el conjunto de entrenamiento y validación
    train_dataloader = torch.utils.data.DataLoader(conjunto_entrenamiento, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    val_dataloader = torch.utils.data.DataLoader(conjunto_validacion, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    return train_dataloader, val_dataloader

# Aplico las funcion de los DataLoaders
train_dataloader, val_dataloader = create_dataloaders(new_train_directory , new_val_directory , BATCH_SIZE, CPU_CORES)

In [ ]:
#Genero una lista de PetIDs con imagen en el orden en que aparecen en el data loader
test_sample_ids = [i[0].split('/')[-1].split('-')[0] for i in val_dataloader.dataset.samples]

### Entrenamiento

In [ ]:
def train_val(model, criterion, dataloaders, datasets, device, num_epochs=2, lr=0.001, momentum = 0.9 ,trial=None):

    # Instancio Stochastic Gradient Descent (SGD): Defino el parámetro del Learning Rate (define "el paso" en que avanzan los pesos en cada iteración) y el Momentum (pone innercia a la dirección del gradiente descendiente para que no cambie de dirección en minimos locales)
    optimizer = optim.SGD(resnet50.parameters(), lr=lr, momentum=momentum) # Parámetros default del SGD

    #Inicializo variables
    since = time.time()


    #Inicializo variable de mejor kappa entre trials
    try:
        #Intento obtener el mejor kappa de optuna
        previous_best = study.best_value
    except:
        #Si no hay, seteo -999
        previous_best = -999

    #Inicializo variables de mejor modelo y mejor accuracy y mejor kappa de este trial
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_kappa =  -999


    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        #Inicializo listas de kappa true y predicted y scores para esta epoch
        epoch_kappa_labels_true = []
        epoch_kappa_labels_predicted = []
        epoch_output_scores = []

        #Cada epoch tiene una fase de entrenamiento y validación
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            #Inicializo variables de loss y accuracy para esta fase de epoch
            epoch_phase_running_loss = 0.0
            epoch_phase_running_corrects = 0

            # Itero sobre los datos.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                # Track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    elif phase == 'val':
                        #Agrego los valores de kappa true y predicted para cada batch en validación
                        epoch_kappa_labels_true.extend(labels.cpu().numpy().tolist())
                        epoch_kappa_labels_predicted.extend(preds.cpu().numpy().tolist())
                        outputs_np = outputs.cpu().numpy()
                        epoch_output_scores.extend([outputs_np[i,:] for i in range(outputs_np.shape[0])])

                # Statistics for each phase
                epoch_phase_running_loss += loss.item() * inputs.size(0)
                epoch_phase_running_corrects += torch.sum(preds == labels.data)

                #END OF BATCH

            epoch_loss = epoch_phase_running_loss / len(datasets[phase])
            epoch_acc = epoch_phase_running_corrects.double() / len(datasets[phase])

            #Calculo el kappa para cada epoch
            if phase == 'train':
                #overall_train_losses.append(epoch_loss)
                current_kappa_score = np.nan
            else:
                #overall_val_losses.append(epoch_loss)
                current_kappa_score = cohen_kappa_score(epoch_kappa_labels_true,
                                  epoch_kappa_labels_predicted,
                                  weights = 'quadratic')



            print(f'{phase.title()} Loss: {epoch_loss:.4f} Acc: {epoch_acc*100:.2f}% Kappa: {current_kappa_score:.3f}')

            # If this is the best Epoch so far -> Deep copy the model
            if phase == 'val' and current_kappa_score > best_kappa:
                best_acc = epoch_acc
                best_kappa = current_kappa_score
                best_model_wts = copy.deepcopy(model.state_dict())


                #Best Epoch within a trial and better than previous trials
                if trial is not None and best_kappa > previous_best:

                    #Save test dataset with predictions
                    predicted_filename = os.path.join(PATH_TO_TEMP_FILES,f'test_{trial.study.study_name}_{trial.number}.joblib')
                    predicted_df = pd.DataFrame({'PetID':test_sample_ids,
                                'pred':epoch_output_scores}).merge(val_data, on='PetID')
                    dump(predicted_df, predicted_filename)

                    #Generate and save CM
                    cm_filename = os.path.join(PATH_TO_TEMP_FILES,f'cm_{trial.study.study_name}_{trial.number}.jpg')
                    plot_confusion_matrix(epoch_kappa_labels_true,epoch_kappa_labels_predicted).write_image(cm_filename)

            #END OF PHASE

        #END OF EPOCH

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:.2f}%'.format(best_acc * 100))

    # Load best model weights
    model.load_state_dict(best_model_wts)

    # Save in optuna trial the best test dataset, cm and model weights
    if trial is not None and best_kappa > previous_best:
        upload_artifact(trial, predicted_filename, artifact_store)

        upload_artifact(trial, cm_filename, artifact_store)

        file_name = f'{MODEL_NAME}_{MODEL_VERSION}_{trial.number}.pth'
        model_path = os.path.join(PATH_TO_TEMP_FILES, file_name)
        torch.save(model, model_path) # Podemos guardar solo los pesos si queremos: best_model.state_dict()
        upload_artifact(trial, model_path, artifact_store)

    return model,best_kappa

best_model,_ = train_val(resnet50, criterion,
                       dataloaders={'train': train_dataloader,
                                    'val': val_dataloader},
                       datasets={'train': train, 'val': test},
                       device=device,
                       num_epochs=4)
# Guardo el modelo
run_id = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
file_name = f'{MODEL_NAME}_{MODEL_VERSION}_{run_id}.pth'
model_path = os.path.join(PATH_TO_TEMP_FILES, file_name)
torch.save(best_model, model_path) # Podemos guardar solo los pesos si queremos: best_model.state_dict()
print(f'Modelo guardado en {model_path}')

Epoch 0/3
----------


100%|██████████| 20/20 [05:25<00:00, 16.27s/it]


Train Loss: 0.1249 Acc: 1.98% Kappa: nan


100%|██████████| 6/6 [02:01<00:00, 20.31s/it]


Val Loss: 0.1346 Acc: 2.97% Kappa: 0.121
Epoch 1/3
----------


...

Val Loss: 0.1295 Acc: 3.00% Kappa: 0.136
Epoch 3/3
----------


100%|██████████| 20/20 [00:19<00:00,  1.01it/s]


Train Loss: 0.1133 Acc: 3.03% Kappa: nan


100%|██████████| 6/6 [00:02<00:00,  2.15it/s]


Val Loss: 0.1285 Acc: 3.17% Kappa: 0.215
Training complete in 8m 35s
Best val Acc: 3.17%
Modelo guardado en /content/drive/MyDrive/LaboII/work/optuna_temp_artifacts/04 ResNet_1.0.0_20250828_224607.pth


### Predicción Imágenes

# **Entrenamiento con optuna**

In [ ]:
import os, torch, optuna
from torchvision.models import resnet50
from optuna.artifacts import FileSystemArtifactStore

# Carpeta de artefactos (asegura que exista)
os.makedirs(PATH_TO_OPTUNA_ARTIFACTS, exist_ok=True)

# Store de artefactos
wartifact_store = FileSystemArtifactStore(base_path=PATH_TO_OPTUNA_ARTIFACTS)

# Alias opcional por si tu train_val usa 'artifact_store'
artifact_store = wartifact_store

def optuna_train(trial):
    epochs   = trial.suggest_int('epochs', 5, 5)
    lr       = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    momentum = trial.suggest_float('momentum', 0.0, 0.95)

    # (opcional) semilla por trial para reproducibilidad
    seed = 42 + trial.number
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    # Entrenamiento / validación
    _, best_score = train_val(
        resnet50,
        criterion,
        dataloaders={'train': train_dataloader, 'val': val_dataloader},
        datasets={'train': train_data, 'val': val_data},
        device=device,
        num_epochs=epochs,
        lr=lr,
        momentum=momentum,
        trial=trial
    )

    return best_score



In [ ]:
# --- imports mínimos que usa train_val ---
import time, copy, numpy as np, pandas as pd
from tqdm.auto import tqdm
from joblib import dump
from sklearn.metrics import cohen_kappa_score
import torch.optim as optim
import torch.nn as nn
from torchvision.models import ResNet50_Weights

# Si tu artifact store se llama 'wartifact_store', lo reuso dentro
artifact_store = wartifact_store  # deja este alias si usás wartifact_store

def train_val(model_ctor,                      # <-- pasa el CONSTRUCTOR (p.ej., resnet50)
              criterion,
              dataloaders: dict,               # {'train': train_dataloader, 'val': val_dataloader}
              datasets: dict,                  # {'train': train_data, 'val': val_data}
              device: str,
              num_epochs: int = 2,
              lr: float = 1e-3,
              momentum: float = 0.9,
              trial=None,
              num_classes: int = 5):

    # instancio el modelo desde el constructor
    model = model_ctor(weights=ResNet50_Weights.IMAGENET1K_V2)
    in_f = model.fc.in_features
    model.fc = nn.Linear(in_f, num_classes)
    model.to(device)

    # OPTIMIZER correcto (sobre EL MODELO, no sobre 'resnet50')
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    since = time.time()

    # mejor kappa previo en Optuna (si existe)
    try:
        previous_best = study.best_value
    except Exception:
        previous_best = -999

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_kappa = -999

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs-1}\n' + '-'*10)

        epoch_kappa_labels_true = []
        epoch_kappa_labels_pred = []
        epoch_output_scores = []

        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in tqdm(dataloaders[phase], leave=False):
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad(set_to_none=True)

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    else:
                        # para kappa + guardar scores
                        epoch_kappa_labels_true.extend(labels.detach().cpu().tolist())
                        epoch_kappa_labels_pred.extend(preds.detach().cpu().tolist())
                        epoch_output_scores.extend(outputs.detach().cpu().numpy().tolist())

                running_loss += loss.item() * inputs.size(0)
                running_corrects += (preds == labels).sum().item()

            epoch_loss = running_loss / len(datasets[phase])
            epoch_acc  = running_corrects / len(datasets[phase])

            if phase == 'val':
                current_kappa = cohen_kappa_score(epoch_kappa_labels_true,
                                                  epoch_kappa_labels_pred,
                                                  weights='quadratic')
            else:
                current_kappa = np.nan

            print(f"{phase.title()} Loss: {epoch_loss:.4f} Acc: {epoch_acc*100:.2f}% Kappa: {current_kappa:.3f}")

            # guardar mejor
            if phase == 'val' and current_kappa > best_kappa:
                best_acc = epoch_acc
                best_kappa = current_kappa
                best_model_wts = copy.deepcopy(model.state_dict())

                # artefactos solo si mejora al mejor de la study
                if trial is not None and best_kappa > previous_best:
                    # test_sample_ids y plot_confusion_matrix pueden no estar definidos; protejo:
                    try:
                        predicted_df = pd.DataFrame({'PetID': test_sample_ids, 'pred': epoch_output_scores}) \
                                       .merge(datasets['val'], on='PetID')
                        predicted_filename = os.path.join(PATH_TO_TEMP_FILES,
                                                          f'test_{trial.study.study_name}_{trial.number}.joblib')
                        dump(predicted_df, predicted_filename)
                        optuna.artifacts.upload_artifact(trial, predicted_filename, artifact_store)
                    except Exception:
                        pass
                    try:
                        cm_filename = os.path.join(PATH_TO_TEMP_FILES,
                                                   f'cm_{trial.study.study_name}_{trial.number}.jpg')
                        # asumimos que tenés plot_confusion_matrix(y_true, y_pred) -> figura PIL/plotly con .write_image
                        plot_confusion_matrix(epoch_kappa_labels_true, epoch_kappa_labels_pred).write_image(cm_filename)
                        optuna.artifacts.upload_artifact(trial, cm_filename, artifact_store)
                    except Exception:
                        pass

        # end epochs

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:.2f}%'.format(best_acc * 100))

    model.load_state_dict(best_model_wts)

    # guardar modelo si superó al mejor de la study
    if trial is not None and best_kappa > previous_best:
        file_name = f'{MODEL_NAME}_{MODEL_VERSION}_{trial.number}.pth'
        model_path = os.path.join(PATH_TO_TEMP_FILES, file_name)
        torch.save(model, model_path)   # también podrías guardar sólo state_dict
        optuna.artifacts.upload_artifact(trial, model_path, artifact_store)

    return model, best_kappa


In [ ]:
print({
  'criterion': 'ok' if 'criterion' in globals() else 'faltante',
  'train_val': 'ok' if 'train_val' in globals() else 'faltante',
  'train_dataloader': 'ok' if 'train_dataloader' in globals() else 'faltante',
  'val_dataloader': 'ok' if 'val_dataloader' in globals() else 'faltante',
})


{'criterion': 'ok', 'train_val': 'ok', 'train_dataloader': 'ok', 'val_dataloader': 'ok'}


In [ ]:
import torch
import torch.nn as nn

# Loss estándar para clasificación multiclase
criterion = nn.CrossEntropyLoss()

print("criterion listo:", criterion)


criterion listo: CrossEntropyLoss()


In [ ]:
# Pre-flight: definir device y datasets a partir de los dataloaders
import torch, os
from urllib.parse import quote

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

# Enlazar datasets si aún no existen en la sesión
if 'train_data' not in globals():
    train_data = getattr(train_dataloader, 'dataset', None)
if 'val_data' not in globals():
    val_data   = getattr(val_dataloader, 'dataset', None)
assert train_data is not None and val_data is not None, "Definí primero train_dataloader/val_dataloader"

# (Opcional) asegurar storage_url si no lo definiste aún
if 'storage_url' not in globals():
    DB_PATH = os.path.join(PROJECT_DIR, "work", "optuna_studies.sqlite3")
    os.makedirs(os.path.dirname(DB_PATH), exist_ok=True)
    storage_url = "sqlite:///" + quote(DB_PATH, safe="/")
    print("storage_url:", storage_url)

# (Opcional) asegurar carpeta de artefactos
os.makedirs(PATH_TO_OPTUNA_ARTIFACTS, exist_ok=True)



device: cuda


In [ ]:
import os, optuna
from urllib.parse import quote

# DB en tu Drive (carpeta work del proyecto) – se crea si no existe
DB_PATH = os.path.join(PROJECT_DIR, "work", "optuna_studies.sqlite3")
os.makedirs(os.path.dirname(DB_PATH), exist_ok=True)

# URL para SQLAlchemy (escapa espacios)
storage_url = "sqlite:///" + quote(DB_PATH, safe="/")

study = optuna.create_study(
    direction='maximize',
    storage=storage_url,
    study_name=f'{MODEL_NAME}_{MODEL_VERSION}',
    load_if_exists=True
)

study.optimize(optuna_train, n_trials=30)

print("Best value:", study.best_value)
print("Best params:", study.best_params)



[I 2025-08-28 22:47:40,305] Using an existing study with name '04 ResNet_1.0.0' instead of creating a new one.


Epoch 0/4
----------


  0%|          | 0/20 [00:00<?, ?it/s]

Train Loss: 1.4938 Acc: 29.50% Kappa: nan


  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Train Loss: 1.5025 Acc: 27.09% Kappa: nan


  0%|          | 0/6 [00:00<?, ?it/s]

Val Loss: 1.4535 Acc: 32.33% Kappa: 0.160
Epoch 1/4
----------


  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
#esto agregue porque me quede sin memoria y se corto en el train 20, lo que hago es arrancar desde donde me quede, osea copie y pegue lo de arriba (puede optimizarlo.. son las 2 am ya no me da la cabeza)

import optuna
study = optuna.create_study(
    direction='maximize',
    storage=storage_url,
    study_name=f'{MODEL_NAME}_{MODEL_VERSION}',
    load_if_exists=True
)
print(len(study.trials), study.best_value, study.best_params)


In [ ]:
import optuna
done = sum(t.state == optuna.trial.TrialState.COMPLETE for t in study.trials)
study.optimize(optuna_train, n_trials=max(0, 30 - done))


In [ ]:
print("storage_url:", storage_url)
print("study_name:", f"{MODEL_NAME}_{MODEL_VERSION}")
import os; print("DB existe?", os.path.exists(DB_PATH), "tamaño:", os.path.getsize(DB_PATH))

print("trials totales:", len(study.trials))
import optuna
from collections import Counter
print(Counter(t.state for t in study.trials))


storage_url: sqlite:////content/drive/MyDrive/LaboII/work/optuna_studies.sqlite3
study_name: 04 ResNet_1.0.0
DB existe? True tamaño: 135168
trials totales: 28
Counter({3: 14, 1: 12, 0: 2})


In [ ]:
import os, torch
from torchvision.models.resnet import ResNet, BasicBlock, Bottleneck

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
torch.serialization.add_safe_globals([ResNet, BasicBlock, Bottleneck])

PATH_TO_TEMP_FILES = "/content/drive/MyDrive/LaboII/work/optuna_temp_artifacts"
MODEL_NAME, MODEL_VERSION, TRIAL_NUM = "04 ResNet", "1.0.0", 13  #cambiar segun el resultado
MODEL_FILE = os.path.join(PATH_TO_TEMP_FILES, f"{MODEL_NAME}_{MODEL_VERSION}_{TRIAL_NUM}.pth")
assert os.path.exists(MODEL_FILE), f"No existe: {MODEL_FILE}"

model = torch.load(MODEL_FILE, map_location=DEVICE, weights_only=False)
model.to(DEVICE).eval()
print("Modelo cargado desde:", MODEL_FILE)


Modelo cargado desde: /content/drive/MyDrive/LaboII/work/optuna_temp_artifacts/04 ResNet_1.0.0_12.pth


In [ ]:
import os
print("Existe?", os.path.exists(MODEL_FILE))
print("Tamaño (bytes):", os.path.getsize(MODEL_FILE))
import torch, torchvision
print("torch:", torch.__version__, "torchvision:", torchvision.__version__)


Existe? True
Tamaño (bytes): 94421825
torch: 2.8.0+cu126 torchvision: 0.23.0+cu126


In [ ]:
### aca mi conseho es que busquen en su drive el nombre del modelo y ponen la ruta

import torch
from torchvision.models.resnet import ResNet, BasicBlock, Bottleneck

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# permitir clases de ResNet para la carga segura
torch.serialization.add_safe_globals([ResNet, BasicBlock, Bottleneck])

# carga el modelo completo
model = torch.load(MODEL_FILE, map_location=DEVICE, weights_only=False)

model.to(DEVICE)
model.eval()

print("Modelo importado como objeto completo en", DEVICE)


Modelo importado como objeto completo en cuda


In [ ]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
best_model = model   # asegurar que el nombre coincide con el que usás abajo
best_model.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
#Evaluar solo-imágenes (validacion) con el mejor modelo

import torch.nn.functional as F

all_preds = []
all_labels = []
all_probabilities = []

best_model.eval()
with torch.no_grad():
    for images, labels in val_dataloader:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        logits = best_model(images)
        probs  = F.softmax(logits, dim=1)

        preds = probs.argmax(1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_probabilities.extend(probs.cpu().numpy())

len(all_preds), len(all_labels), len(all_probabilities)



(1210, 1210, 1210)

In [ ]:
#Matriz de confusión y Kappa
display(plot_confusion_matrix(all_labels, all_preds))
from sklearn.metrics import cohen_kappa_score
print("Kappa (val, imágenes):", cohen_kappa_score(all_labels, all_preds, weights='quadratic'))


Kappa (val, imágenes): 0.05900911951925336


## Combinación de Resultados

In [ ]:
# Dataset con resultados
Resultados = test[["PetID", "AdoptionSpeed"]]

# Sumamos la clase predicha por los datos tabulares
Resultados["Pred_DT"] = lgb_model.predict(X_test).argmax(axis=1)

# Sumamos las probabilidades predichas por los tabulares
df_pred = pd.DataFrame(lgb_model.predict(X_test), columns=[f"DT_Clase_{i}" for i in range(5)])
df_pred["PetID"] = test["PetID"].values

Resultados = pd.merge(Resultados, df_pred, on = "PetID", how="inner")

# Se crea un data frame con los resultados de las imágenes
pred_Im = pd.DataFrame({
    "PetID" : [s[-9:] for s in test_sample_ids],
    "Im_Pred" : all_preds,
    "Im_Clase_0" : [p[0] for p in all_probabilities],
    "Im_Clase_1" : [p[1] for p in all_probabilities],
    "Im_Clase_2" : [p[2] for p in all_probabilities],
    "Im_Clase_3" : [p[3] for p in all_probabilities],
    "Im_Clase_4" : [p[4] for p in all_probabilities]
})

#Sumamos esas predicciones al dataset Resultados
Resultados = pd.merge(Resultados, pred_Im, on = "PetID", how="left")


/tmp/ipython-input-1549435372.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# Calcular los promedios y crear 5 nuevas columnas
for i in range(5):
    Resultados[f'Promedio_Clase_{i}'] = (Resultados[f'DT_Clase_{i}'] + Resultados[f'Im_Clase_{i}']) / 2

# Realizar la prediccion con las 5 columnas promedio
columnas_promedio = [f'Promedio_Clase_{i}' for i in range(5)]

Resultados['Pred_Promedio'] = Resultados[columnas_promedio].idxmax(axis=1)

# A los que no son NA se los convierte a número
Resultados.loc[Resultados['Pred_Promedio'].notna(), 'Pred_Promedio'] = Resultados.loc[Resultados['Pred_Promedio'].notna(), 'Pred_Promedio'].str.split('_').str.get(-1).astype(int)

# A los que son NA porque no está la predicción de la imagen se les asigna la predicción de DT
Resultados.loc[Resultados['Pred_Promedio'].isna(), 'Pred_Promedio'] = Resultados.loc[Resultados['Pred_Promedio'].isna(), 'Pred_DT']
Resultados['Pred_Promedio'] = Resultados['Pred_Promedio'].astype('int64')
# Mostrar el DataFrame final con las nuevas columnas ---
Resultados

/tmp/ipython-input-3673304729.py:8: FutureWarning:

The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError



,PetID,AdoptionSpeed,Pred_DT,DT_Clase_0,DT_Clase_1,DT_Clase_2,DT_Clase_3,DT_Clase_4,Im_Pred,Im_Clase_0,Im_Clase_1,Im_Clase_2,Im_Clase_3,Im_Clase_4,Promedio_Clase_0,Promedio_Clase_1,Promedio_Clase_2,Promedio_Clase_3,Promedio_Clase_4,Pred_Promedio
0,06bfadf29,4,4,0.019093,0.080874,0.241954,0.206344,0.451736,1.0,0.085920,0.234837,0.232623,0.218575,0.228045,0.052506,0.157855,0.237289,0.212459,0.339890,4
1,00b23513e,2,2,0.019147,0.244582,0.493757,0.160160,0.082355,3.0,0.127321,0.243232,0.168386,0.246219,0.214842,0.073234,0.243907,0.331071,0.203189,0.148598,2
2,40c0fdccb,4,2,0.010258,0.221100,0.346915,0.183360,0.238367,4.0,0.099822,0.192025,0.228894,0.214348,0.264911,0.055040,0.206563,0.287905,0.198854,0.251639,2
3,e748f52b7,2,2,0.035306,0.086538,0.410106,0.314620,0.153431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
4,e0e604fc2,1,2,0.031680,0.210089,0.370838,0.221153,0.166241,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2994,48c598ef0,1,1,0.012033,0.453062,0.321243,0.157340,0.056323,4.0,0.091543,0.236424,0.221551,0.206088,0.244394,0.051788,0.344743,0.271397,0.181714,0.150358,1
2995,aa6b8a52b,2,1,0.010569,0.337649,0.291667,0.257241,0.102873,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2996,2bbca459e,4,4,0.042007,0.228973,0.223579,0.123580,0.381861,2.0,0.099823,0.160779,0.259899,0.224754,0.254745,0.070915,0.194876,0.241739,0.174167,0.318303,4
2997,6cb5e1b06,1,2,0.010404,0.299551,0.375792,0.184774,0.129479,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [ ]:
#Muestro matriz de confusion y kappa
display(plot_confusion_matrix(Resultados["AdoptionSpeed"].values, Resultados["Pred_Promedio"].values))

cohen_kappa_score(Resultados["AdoptionSpeed"].values, Resultados["Pred_Promedio"].values,
                             weights = 'quadratic')

np.float64(0.3721125670514528)

In [ ]:
# Se crea un data frame con los resultados de las imágenes
pred_Im = pd.DataFrame({
    "PetID" : [s[-9:] for s in test_sample_ids],
    "Im_Pred" : all_preds,
    "Im_Clase_0" : [p[0] for p in all_probabilities],
    "Im_Clase_1" : [p[1] for p in all_probabilities],
    "Im_Clase_2" : [p[2] for p in all_probabilities],
    "Im_Clase_3" : [p[3] for p in all_probabilities],
    "Im_Clase_4" : [p[4] for p in all_probabilities]
})

pred_Im.to_csv("pred_imagenes.csv", index=False)

In [ ]:

import os

# Carpeta de destino en tu Drive (cambiá si querés otra)
DEST = "/content/drive/MyDrive/LaboII/work"
os.makedirs(DEST, exist_ok=True)

# Aseguro PetID como texto, por las dudas
pred_Im = pred_Im.copy()
pred_Im["PetID"] = pred_Im["PetID"].astype(str)

# Guardar EXACTAMENTE el mismo DataFrame a CSV
out_path = os.path.join(DEST, "pred_imagenes.csv")
pred_Im.to_csv(out_path, index=False, encoding="utf-8")

print("✅ CSV guardado en:", out_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ CSV guardado en: /content/drive/MyDrive/LaboII/work/pred_imagenes.csv


In [ ]:
import numpy as np
from sklearn.metrics import cohen_kappa_score

# 1) Predicción de ensamble (promedio) sin idxmax y con fallback a tabular
cols = [f'Promedio_Clase_{i}' for i in range(5)]
M = Resultados[cols].to_numpy()

has_img = ~np.isnan(M).all(axis=1)               # filas con probs de imagen
pred_prom = Resultados['Pred_DT'].to_numpy()     # fallback: tabular
pred_prom[has_img] = np.nanargmax(M[has_img], axis=1)

Resultados['Pred_Promedio'] = pred_prom.astype(int)

# 2) Kappas
k_tab = cohen_kappa_score(Resultados['AdoptionSpeed'], Resultados['Pred_DT'],
                          weights='quadratic')
k_img = cohen_kappa_score(Resultados.loc[has_img,'AdoptionSpeed'],
                          Resultados.loc[has_img,'Im_Pred'], weights='quadratic')
k_ens = cohen_kappa_score(Resultados['AdoptionSpeed'], Resultados['Pred_Promedio'],
                          weights='quadratic')

print(f"Kappa tabular:   {k_tab:.3f}")
print(f"Kappa imágenes*: {k_img:.3f}  (*solo en filas con imagen)")
print(f"Kappa ensamble:  {k_ens:.3f}")


# Text Classification

In [ ]:
# Data processing
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
import copy

import time
import datetime

from sklearn.metrics import confusion_matrix, cohen_kappa_score

from datasets import Dataset,  DatasetDict

import optuna
from optuna.artifacts import FileSystemArtifactStore, upload_artifact

# Modeling
import torch
from torch.utils.data import DataLoader
from transformers import DistilBertTokenizerFast, DataCollatorWithPadding, AutoModelForSequenceClassification, AdamW, get_scheduler

# Progress bar
from tqdm.auto import tqdm

from utils import plot_confusion_matrix, get_artifact_filename

from joblib import load, dump

# Verificamos que CUDA está funcional
torch.cuda.is_available()

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')